# Scrape the top 25 movies from  a web page

In [ ]:
# import the necessary required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3

In [ ]:
# Initialize variables, data source path, data destination path, database name and table name
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films' # link to the data source page
csvpath = 'D:/Data Engineerring/scraping_project/target_file.csv' # csv file to export the extracted and transformed web data
db_name = 'movies.db' # sqlite database name for the final data set
table_name = 'top_25' # sqlite table to store the csv data
df = pd.DataFrame(columns=["Film","Year","Rotten Tomatoes' Top 100"]) # define an empty dataframe
count = 0 # counter to allow looping through the web page contents

In [ ]:
html_page = requests.get(url).text # get the web page using request library as a text file
data = BeautifulSoup(html_page, 'html.parser') # use BeatifulSoup library to parse the html elements and tags

In [ ]:
tables = data.find_all('tbody') # Find all tables in the web page
rows = tables[0].find_all('tr') # Find all the rows contained in the first table in the web page

In [ ]:
# Find all the data contained in the columns and do transformation
for row in rows:
    if count <= 25:
        col = row.find_all('td') # loop through all the rows to find the tag element td
        if len(col)!= 0 and col[3].contents[0] != 'unranked': # check if all the td elements are not empty and column 3 does not contain the string 'unranked'
            data_dict = {"Film":str(col[1].contents[0]),
                    "Year":int(col[2].contents[0]),
                    "Rotten Tomatoes' Top 100":int(col[3].contents[0])} # extract the required columns beginning with the first row for every column
            df1 = pd.DataFrame(data_dict, index=[0]) # Create a data frame df1 to hold the data extracted above and set the index to begin at zero
            df = pd.concat([df,df1], ignore_index=True) # combine the initial empty data frame (df) with above data frame (df1) 
            df = df.loc[df['Year'] >= 2000] # select movies that were released in the year 2000 and above
    count+=0 # increment the count by 1 and loop through the above code untill the last value 25
   
print(df) # print the data frame

In [ ]:
# Export the data frame to a csv file
df.to_csv(csvpath)

In [ ]:
# Export the csv file to sqlite3 database
conn = sqlite3.connect(db_name)
df.to_sql(table_name,conn,if_exists='replace',index=False)
conn.close